<a href="https://colab.research.google.com/github/sivasaiyadav8143/Python/blob/master/Closure_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Closure Applications

In this example we are going to build an averager function that can average multiple values.

We want to simply be able to feed numbers to that function and get a running average over time, not average a list which requires performing the same calculations (sum and count) over and over again.

In [ ]:
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [ ]:
a = Averager()

In [ ]:
a.add(10)

10.0

In [ ]:
a.add(20)

15.0

In [ ]:
a.add(30)

20.0

We can do this using a closure as follows:

In [ ]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [ ]:
a = averager()

In [ ]:
a(10)

10.0

In [ ]:
a(20)

15.0

In [ ]:
a(30)

20.0

In [ ]:
b = averager()

In [ ]:
b(10)

10.0

In [ ]:
b(30)

20.0

In [ ]:
a.__closure__ , b.__closure__

((<cell at 0x7f1fba8374c8: list object at 0x7f1fba894f48>,),
 (<cell at 0x7f1fba837948: list object at 0x7f1fb9ff1588>,))

Now, instead of storing a list and reclaculating `total` and `count` every time to get the new average, we are going to store the running total and count and update each value each time a new value is added to the running average, and then return `total / count`.

Let's start with a class approach first, where we will use instance variables to store the running total and count and provide an instance method to add a new number and return the current average.

In [ ]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [ ]:
a = Averager()

In [ ]:
a.add(10)

10.0

In [ ]:
a.add(20)

15.0

In [ ]:
a.add(30)

20.0

Now, let's see how we might use a closure to achieve the same thing.

In [ ]:
def averager():
    total = 0
    count = 0
    
    def add(value):
        total += value
        count += 1
        return 0 if count == 0 else total / count
    
    return add
        

In [ ]:
a = averager()

In [ ]:
a.__code__.co_freevars

()

The output showed zero free variables, but closure has two (total and count), zero coz the nested func has two assignments for total and count,now they are local variables to add(), not free variables anymore, inorder to fix this we should use nonlocal keyword to make them freevariables.

In [ ]:
def averager():
    total = 0
    count = 0
    
    def add(value):
        nonlocal total, count
        total += value
        count += 1
        return 0 if count == 0 else total / count
    
    return add

In [ ]:
a = averager()

In [ ]:
a.__code__.co_freevars

('count', 'total')

In [ ]:
a.__closure__

(<cell at 0x7f1fb2329f18: int object at 0xa68aa0>,
 <cell at 0x7f1fb2329c18: int object at 0xa68aa0>)

In [ ]:
a(10)

10.0

In [ ]:
a(20)

15.0

In [ ]:
a(30)

20.0

#### Generalizing this example

We saw that we were essentially able to convert a class to an equivalent functionality using closures. This is actually true in a much more general sense - very often, classes that define a single method (other than initializers) can be implemented using a closure instead.

Let's look at another example of this.

Suppose we want something that can keep track of the running elapsed time in seconds.

In [ ]:
from time import perf_counter

In [ ]:
class Timer:
    def __init__(self):
        self._start = perf_counter()
    
    def __call__(self):
        return (perf_counter() - self._start)

In [ ]:
a = Timer()

Now wait a bit before running the next line of code:

In [ ]:
a()

0.011695334544051804

Let's start another "timer":

In [ ]:
b = Timer()

In [ ]:
print(a())
print(b())

0.03528294403966765
0.011656054820407689


Now let's rewrite this using a closure instead:

In [ ]:
def timer():
    start = perf_counter()
    
    def elapsed():
        # we don't even need to make start nonlocal 
        # since we are only reading it
        return perf_counter() - start
    
    return elapsed

In [ ]:
x = timer()

In [ ]:
x()

0.011068213438975016

In [ ]:
y = timer()

In [ ]:
print(x())
print(y())

0.03419096772236116
0.01164738619174141


In [ ]:
print(a())
print(b())
print(x())
print(y())

0.10822159832175349
0.08475345336494494
0.0462381944113351
0.023573252079387305


Let's write a small function that can increment a counter for us - we don't have an incrementor in Python (the ++ operator in Java or C++ for example):

In [ ]:
def counter(initial_value):
    # initial_value is a local variable here
    
    def inc(increment=1):
        nonlocal initial_value
        # initial_value is a nonlocal (captured) variable here
        initial_value += increment
        return initial_value
    
    return inc

In [ ]:
counter1 = counter(0)

In [ ]:
print(counter1(0))

0


In [ ]:
print(counter1())

1


In [ ]:
print(counter1(10))

11


In [ ]:
counter2 = counter(1000)

In [ ]:
print(counter2(0))

1000


In [ ]:
print(counter2(1))

1001


As you can see, each closure maintains a reference to the initial_value variable that was created when the counter function was called - each time that function was called, a new local variable initial_value was created (with a value assigned from the argument), and it became a nonlocal (captured) variable in the inner scope.

Let's write a function something that can run, and maintain a count of how many times we have run some function.

In [31]:
def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [32]:
def add(a, b):
    return a + b

In [33]:
counted_add = counter(add)

In [34]:
#  the free variables are:
counted_add.__code__.co_freevars 

('cnt', 'fn')

In [35]:
counted_add(1, 2)

add has been called 1 times


3

In [36]:
counted_add(2, 3)

add has been called 2 times


5

In [37]:
def mult(a, b, c):
    return a * b * c

In [38]:
counted_mult = counter(mult)

In [40]:
counted_mult.__code__.co_freevars 

('cnt', 'fn')

In [41]:
counted_mult.__closure__

(<cell at 0x7f1fb2300318: int object at 0xa68ac0>,
 <cell at 0x7f1fb23002e8: function object at 0x7f1fb2304ae8>)

In [39]:
counted_mult(1, 2, 3)

mult has been called 1 times


6

Let's write a function that actually store the function name and the number of calls in a global dictionary instead of just printing it out all the time.

In [44]:
counters = dict()

def counter(fn):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        global counter
        counters[fn.__name__] = cnt  # counters is global
        return fn(*args, **kwargs)
    
    return inner

In [45]:
counted_add = counter(add)
counted_mult = counter(mult)

counters is a global variable, and therefore not a free variable:

In [47]:
counted_add.__code__.co_freevars, counted_mult.__code__.co_freevars

(('cnt', 'fn'), ('cnt', 'fn'))

In [48]:
counted_add(1, 2)

3

In [49]:
counted_add(2, 50)

52

In [52]:
counted_mult(1, 2, 'python')

'pythonpython'

In [51]:
counted_mult(2, 3, 'b')

'bbbbbb'

In [53]:
print(counters)

{'add': 2, 'mult': 3}


Instead of creating the counters global variable first and making sure we are naming it that way, so instead, we're going to pass it as an argument to the counter function:

In [54]:
def counter(fn, counters):
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        counters[fn.__name__] = cnt  # counters is nonlocal
        return fn(*args, **kwargs)
    
    return inner

In [55]:
func_counters = dict()
counted_add = counter(add, func_counters)
counted_mult = counter(mult, func_counters)

In [56]:
counted_add.__code__.co_freevars

('cnt', 'counters', 'fn')

counters is now a free variable.

Lets write a small factorial function.

In [57]:
def fact(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [58]:
fact(5)

120

In [59]:
fact(4)

24

In [60]:
fact = counter(fact, func_counters)

In [63]:
fact.__code__.co_freevars

('cnt', 'counters', 'fn')

In [64]:
fact.__closure__

(<cell at 0x7f1fb2300228: int object at 0xa68ae0>,
 <cell at 0x7f1fb23007f8: dict object at 0x7f1fb2305fc0>,
 <cell at 0x7f1fb2300b28: function object at 0x7f1fb22ffd90>)

In [61]:
fact(5)

120

In [62]:
fact(4)

24

In [65]:
print(func_counters)

{'fact': 2}


we have essentially added some functionality to our fact function, without modifying what the fact function actually returns.